<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/reading_the_chords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


%%capture
#@title Set Up
!git clone https://github.com/GenaroHacker/write_imports.git
from write_imports.write_imports import write_imports

!git clone https://github.com/GenaroHacker/creating_chord_collection.git


from creating_chord_collection.collection.resources.scales import scales
from creating_chord_collection.collection.resources.transposable_figures import transposable_figures





In [2]:

# Delete this line to see the new imports
%%capture



#@title Import Statements
#Modules: ['creating_chord_collection', 'write_imports']




from creating_chord_collection.builder.director import Director

from creating_chord_collection.builder.builders import AbstractBuilder
from creating_chord_collection.builder.builders import ShortBuilder
from creating_chord_collection.builder.builders import LongBuilder

from creating_chord_collection.collection.chord import GuitarChord

from creating_chord_collection.collection.collection import ChordCollection



from write_imports.write_imports import write_imports

from write_imports.LearningPython.BasicExamples.intro_functions import print_hello_world





print(write_imports([]))


In [3]:
# @title txt to dict
def parse_chords(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    chords = {}
    current_chord = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Check if the line is a chord name (no spaces) or a chord position (contains spaces)
        if ' ' in line:
            if current_chord:
                chords[current_chord].append(line)
        else:
            current_chord = line
            chords[current_chord] = []

    return chords

# Example usage
file_path = '/content/creating_chord_collection/chords.txt'  # Replace with your file path
chord_dict = parse_chords(file_path)
print(chord_dict['A'][0:2])
print(chord_dict['Gbmmaj11/A'][0:2])


['x 0 2 2 2 0', '5 7 7 6 5 5']
['x 0 3 2 0 2', '5 4 4 4 6 4']


In [4]:
# @title Split by / character
def process_keys(my_dict):
    absolute_keys_dict = {}
    left_relative_keys_dict = {}
    right_relative_keys_dict = {}

    for key, value in my_dict.items():
        if '/' in key:
            parts = key.split('/')
            if len(parts) != 2:
                raise ValueError("Key contains more than one '/' character: " + key)
            left_relative_keys_dict[parts[0]] = value
            right_relative_keys_dict[parts[1]] = value
        else:
            absolute_keys_dict[key] = value

    return absolute_keys_dict, left_relative_keys_dict, right_relative_keys_dict

# Example usage
my_dict = chord_dict

try:
    abs_keys_dict, left_rels_dict, right_rels_dict = process_keys(my_dict)
    #print("Absolute Keys Dict:", abs_keys_dict)
    #print("Left Parts of Relative Keys Dict:", left_rels_dict)
    #print("Right Parts of Relative Keys Dict:", right_rels_dict)
except ValueError as e:
    print("Error:", e)


In [6]:
# @title Merge into one dict
def merge_dicts(dict_list):
    merged_dict = {}
    for d in dict_list:
        for key, value in d.items():
            if key not in merged_dict:
                merged_dict[key] = set()
            merged_dict[key].update(value)

    # Convert sets back to lists
    for key in merged_dict:
        merged_dict[key] = list(merged_dict[key])

    return merged_dict

# Example usage
dict_list = [abs_keys_dict, left_rels_dict, right_rels_dict]

merged = merge_dicts(dict_list)
print(merged["G"])


['x 11 9 12 x 10', '3 2 5 x 3 x', 'x x 6 4 3 3', '3 2 4 1 x 1', '3 4 4 4 6 4', '4 x 5 4 3 3', 'x 10 9 7 x 10', '4 5 5 7 x 7', 'x 11 12 12 12 x', 'x x 5 7 8 7', '3 2 x 4 3 x', 'x 10 9 12 x 10', 'x 10 9 7 8 7', '4 5 x 4 3 3', '3 5 5 4 3 3', '4 2 5 x 3 3', 'x 10 9 x 12 10', '3 2 0 0 0 3', '15 14 12 12 12 15', '4 5 5 4 x 7', 'x 10 12 12 12 10', '4 2 x 4 3 3', '3 x 3 1 0 2', 'x 10 9 x 8 10', 'x 11 9 x 8 10', '3 2 0 0 3 3', 'x 11 x 12 12 10', '15 14 12 12 15 x', 'x x 6 7 8 7']


In [10]:
# @title Update dict
def convert_and_invert(input_string):
    elements = input_string.split()
    converted_elements = [None if el == 'x' else int(el) for el in elements]
    inverted_elements = converted_elements[::-1]
    return inverted_elements

def process_dict(input_dict):
    processed_dict = {}
    for key, value in input_dict.items():
        # Apply convert_and_invert to each string in the list
        processed_dict[key] = [convert_and_invert(s) for s in value]

    return processed_dict

# Example usage
my_dict = merged

processed_dict = process_dict(my_dict)
print(processed_dict["G"])


[[10, None, 12, 9, 11, None], [None, 3, None, 5, 2, 3], [3, 3, 4, 6, None, None], [1, None, 1, 4, 2, 3], [4, 6, 4, 4, 4, 3], [3, 3, 4, 5, None, 4], [10, None, 7, 9, 10, None], [7, None, 7, 5, 5, 4], [None, 12, 12, 12, 11, None], [7, 8, 7, 5, None, None], [None, 3, 4, None, 2, 3], [10, None, 12, 9, 10, None], [7, 8, 7, 9, 10, None], [3, 3, 4, None, 5, 4], [3, 3, 4, 5, 5, 3], [3, 3, None, 5, 2, 4], [10, 12, None, 9, 10, None], [3, 0, 0, 0, 2, 3], [15, 12, 12, 12, 14, 15], [7, None, 4, 5, 5, 4], [10, 12, 12, 12, 10, None], [3, 3, 4, None, 2, 4], [2, 0, 1, 3, None, 3], [10, 8, None, 9, 10, None], [10, 8, None, 9, 11, None], [3, 3, 0, 0, 2, 3], [10, 12, 12, None, 11, None], [None, 15, 12, 12, 14, 15], [7, 8, 7, 6, None, None]]


In [5]:
['Db', 'Eb', 'Gb', 'Ab', 'Bb', 'C#', 'D#', 'F#', 'G#', 'A#', 'C', 'D', 'E', 'F', 'G', 'A', 'B']

['Db',
 'Eb',
 'Gb',
 'Ab',
 'Bb',
 'C#',
 'D#',
 'F#',
 'G#',
 'A#',
 'C',
 'D',
 'E',
 'F',
 'G',
 'A',
 'B']